In [1]:
# Библиотеки
import csv
from sklearn.pipeline import Pipeline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction import DictVectorizer
from collections import defaultdict

In [2]:
# Извлечение слова из строки вида 'word#X:Y'
def lexeme(str):
    """Возвращает word, если вид word#X
    Возвращает word, freq, если вид word#X:freq
    """
    if '#' in str:
        word, tail = str.split('#', 1)
    else:
        word, tail = str, None

    if tail:
        if ':' in tail:
            labels, tail = tail.split(':', 1)
        else:
            labels, tail = tail, None

    if tail:
        freq = float(tail)
    else:
        freq = 1
    
    return word, freq

In [3]:
synsets = {}  # Словарь {id -> список синсетов}
index = defaultdict(list)  # Словарь {слово -> номера синсетов с упоминаниями}
lexicon = set()  # Набор всех слов в базе
relations = {}  # Словарь {id -> словарь близких слов}

# Считываем файл
with open('watset-mcl-mcl-joint-exp-linked.tsv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter='\t', quoting=csv.QUOTE_NONE)  

    # Перебираем строки и заполняем переменные synsets и relations словами.
    # Ключи - номер строки (с единицы).
    # Значения - словари вида {слово -> частота}.
    for row in reader:
        synsets_dict = dict()
        for word in row[2].split(', '):
            if word:
                key, value = lexeme(word)
                synsets_dict[key] = value
        synsets[int(row[0])] = synsets_dict
        
        relations_dict = dict()
        for word in row[4].split(', '):
            if word:
                key, value = lexeme(word)
                relations_dict[key] = value
        relations[int(row[0])] = relations_dict

        # Закидываем номер строки в index для каждого слова.
        for word in synsets[int(row[0])]:
            index[word].append(int(row[0]))

        # Обновляем лексикон базы данных
        lexicon.update(synsets[int(row[0])])

In [4]:
v = Pipeline([('dict', DictVectorizer()), ('tfidf', TfidfTransformer())])
v.fit(synsets.values())

Pipeline(steps=[('dict', DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
        sparse=True)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True))])

In [5]:
# Преобразование предложения в "рабочий" вид.
def formatting(sentence, word):
    """Возвращает словарь слов, из которых состоит предложение."""
    table = {ord(char): None for char in '!?,.;'}  # Удаляем посторонние символы
    sentence = sentence.translate(table)
    sentence = sentence.lower()
    
    words_in_sentence = [word for word in sentence.split(' ') if word] 
    words_in_sentence.remove(word)  # Удаляем исходное слово из списка соседних по предложению слов.
    words_dict = dict()
    for word in words_in_sentence:
         words_dict[word] = 1
    
    return words_dict

In [6]:
def cos_similar(sentence_dict, word):
    cos_result = dict()
    for i in range(1,(len(synsets))):
        synset = synsets[i]
        if word in synset.keys():
            sim = cosine_similarity(v.transform(synset), v.transform(sentence_dict)).item(0)
            if (sim != 0):
                cos_result[i] = sim
    return cos_result

In [7]:
sentence = (
    'Если подросток добрый, уступчивый и хороший, '
    + 'то это часто воспринимается окружающими как проявление слабости его характера, '
    + 'как неспособность чётко высказать свою позицию.')
word = 'уступчивый'
words_dict = formatting(sentence, word)
result = cos_similar(words_dict, word)
print(result)

{2: 0.024169463991165787}
